In [ ]:
pip install ucimlrepo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X_data = adult.data.features
y_data = adult.data.targets

data = pd.concat([X_data, y_data], axis=1)
data[data == '?'] = np.nan
for col in ['workclass', 'occupation', 'native-country']:
    data[col].fillna(data[col].mode()[0], inplace=True)
data['income']=data['income'].map({'<=50K': '<=50K', '<50K': '<=50K', '<=50K.': '<=50K', '<50K.': '<=50K' , '>50K': '>50K', '>=50K': '>50K', '>50K.': '>50K', '>=50K.': '>50K'})
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [ ]:
data2=data.copy()

In [ ]:
from sklearn.preprocessing import LabelEncoder
categorical = ['workclass','education', 'marital-status', 'occupation', 'relationship',
               'race', 'sex','native-country', 'income']
encoded_data = data.copy()
label_encoder = LabelEncoder()
for col in categorical:
    label_encoder.fit(data[col])
    encoded_data[col] = label_encoder.transform(encoded_data[col])
encoded_data.head()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,6,77516,9,13,4,0,1,4,1,2174,0,40,38,0
1,50,5,83311,9,13,2,3,0,4,1,0,0,13,38,0
2,38,3,215646,11,9,0,5,1,4,1,0,0,40,38,0
3,53,3,234721,1,7,2,5,0,2,1,0,0,40,38,0
4,28,3,338409,9,13,2,9,5,2,0,0,0,40,4,0


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
# Remove potentially biased features
# data = encoded_data.drop(['race', 'sex'], axis=1)

# Address imbalance in the target variable ('income') through downsampling
# Split the dataset by the target variable
lower_income = encoded_data[encoded_data['income'] == 0]
higher_income = encoded_data[encoded_data['income'] == 1]

# Downsample the majority class
lower_income_downsampled = resample(lower_income,
                                    replace=False,    # sample without replacement
                                    n_samples=len(higher_income),     # match minority class size
                                    random_state=108) # reproducible results

# Combine minority class with downsampled majority class
balanced_data = pd.concat([lower_income_downsampled, higher_income])

# Note: This is a basic approach. Consider more sophisticated methods for production.

# Proceed with your analysis on 'balanced_data'
balanced_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
47098,40,3,132222,11,9,4,7,1,4,1,0,0,40,38,0
8782,21,3,494638,7,12,4,9,3,4,1,0,0,15,38,0
62,48,3,149640,11,9,2,13,0,4,1,0,0,40,38,0
8197,23,3,184255,11,9,2,11,5,4,0,0,0,40,38,0
24823,44,1,56651,9,13,2,3,0,4,1,0,0,52,38,0


Splitting the balanced data below

In [ ]:
from sklearn.model_selection import train_test_split
x = balanced_data[['workclass','education', 'marital-status', 'occupation', 'relationship',
               'race', 'sex','native-country', 'age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']]
y = balanced_data['income']

x_train_b, x_test_b, y_train_b, y_test_b = train_test_split(x, y, test_size = 0.3, random_state = 108)

Splitting the unbalanced(Original) data below

In [ ]:
from sklearn.model_selection import train_test_split
x = encoded_data[['workclass','education', 'marital-status', 'occupation', 'relationship',
               'race', 'sex','native-country', 'age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']]
y = encoded_data['income']

x_train_e, x_test_e, y_train_e, y_test_e = train_test_split(x, y, test_size = 0.3, random_state = 108)

Running regression on original unbalanced data below

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Create a logistic regression model within a pipeline that includes scaling
model_e = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, random_state = 108))

# Train the model
model_e.fit(x_train_e, y_train_e)

from sklearn.metrics import accuracy_score, classification_report

# Generate predictions
y_pred_e = model_e.predict(x_test_e)

# Evaluate accuracy
accuracy_e = accuracy_score(y_test_e, y_pred_e)
print(f"Accuracy: {accuracy_e}")

# Detailed performance report
report_e = classification_report(y_test_e, y_pred_e)
print(report_e)

Accuracy: 0.799699720193817
              precision    recall  f1-score   support

           0       0.81      0.95      0.88     11096
           1       0.69      0.32      0.43      3557

    accuracy                           0.80     14653
   macro avg       0.75      0.64      0.66     14653
weighted avg       0.78      0.80      0.77     14653



Running regression on balanced data below

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Create a logistic regression model within a pipeline that includes scaling
model_b = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, random_state = 108))

# Train the model
model_b.fit(x_train_b,y_train_b)

from sklearn.metrics import accuracy_score, classification_report

# Generate predictions
y_pred_b = model_b.predict(x_test_b)

# Evaluate accuracy
accuracy_b = accuracy_score(y_test_b, y_pred_b)
print(f"Accuracy: {accuracy_b}")

# Detailed performance report
report_b = classification_report(y_test_b, y_pred_b)
print(report_b)

Accuracy: 0.7359190075573934
              precision    recall  f1-score   support

           0       0.74      0.72      0.73      3481
           1       0.73      0.75      0.74      3532

    accuracy                           0.74      7013
   macro avg       0.74      0.74      0.74      7013
weighted avg       0.74      0.74      0.74      7013



Removing the sensitive columns from original data and splitting it

In [ ]:
from sklearn.model_selection import train_test_split
x = encoded_data[['workclass','education', 'marital-status', 'occupation', 'relationship',
                   'native-country', 'age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']]
y = encoded_data['income']

x_train_e_sr, x_test_e_sr, y_train_e_sr, y_test_e_sr = train_test_split(x, y, test_size = 0.3, random_state = 108)

Running regression on original unbalanced data and sensitive columns removed below

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Create a logistic regression model within a pipeline that includes scaling
model_e_sr = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, random_state = 108))

# Train the model
model_e_sr.fit(x_train_e_sr,y_train_e_sr)

from sklearn.metrics import accuracy_score, classification_report

# Generate predictions
y_pred_e_sr = model_e_sr.predict(x_test_e_sr)

# Evaluate accuracy
accuracy_e_sr = accuracy_score(y_test_e_sr, y_pred_e_sr)
print(f"Accuracy: {accuracy_e_sr}")

# Detailed performance report
report_e_sr = classification_report(y_test_e_sr, y_pred_e_sr)
print(report_e_sr)

Accuracy: 0.8001091926567938
              precision    recall  f1-score   support

           0       0.81      0.96      0.88     11096
           1       0.70      0.31      0.43      3557

    accuracy                           0.80     14653
   macro avg       0.76      0.63      0.65     14653
weighted avg       0.78      0.80      0.77     14653



Removing the sensitive columns from balanced data and splitting it

In [ ]:
from sklearn.model_selection import train_test_split
x = balanced_data[['workclass','education', 'marital-status', 'occupation', 'relationship',
                   'native-country', 'age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']]
y = balanced_data['income']

x_train_b_sr, x_test_b_sr, y_train_b_sr, y_test_b_sr = train_test_split(x, y, test_size = 0.3, random_state = 108)

Running regression on balanced data and sensitive columns removed below

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Create a logistic regression model within a pipeline that includes scaling
model_b_sr = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, random_state = 108))

# Train the model
model_b_sr.fit(x_train_b_sr,y_train_b_sr)

from sklearn.metrics import accuracy_score, classification_report

# Generate predictions
y_pred_b_sr = model_b_sr.predict(x_test_b_sr)

# Evaluate accuracy
accuracy_b_sr = accuracy_score(y_test_b_sr, y_pred_b_sr)
print(f"Accuracy: {accuracy_b_sr}")

# Detailed performance report
report_b_sr = classification_report(y_test_b_sr, y_pred_b_sr)
print(report_b_sr)

Accuracy: 0.7307856837302154
              precision    recall  f1-score   support

           0       0.73      0.72      0.73      3481
           1       0.73      0.74      0.73      3532

    accuracy                           0.73      7013
   macro avg       0.73      0.73      0.73      7013
weighted avg       0.73      0.73      0.73      7013



Now lets do some fairness metric calculation on all 4 cases

In [ ]:
pip install fairlearn

## Calculation on original unbalanced data below

In [ ]:
from fairlearn.metrics import demographic_parity_ratio
print(demographic_parity_ratio(y_test_e,
                               y_pred_e,
                               sensitive_features=x_test_e['race']))

0.347004959058932


In [ ]:
from fairlearn.metrics import demographic_parity_difference
print(demographic_parity_difference(y_test_e,
                               y_pred_e,
                               sensitive_features=x_test_e['race']))

0.07786771844993991


In [ ]:
from fairlearn.metrics import equalized_odds_ratio
print(equalized_odds_ratio(y_test_e,
                           y_pred_e,
                           sensitive_features=x_test_e['race']))

0.16335645229087853


In [ ]:
from fairlearn.metrics import equalized_odds_difference
print(equalized_odds_difference(y_test_e,
                           y_pred_e,
                           sensitive_features=x_test_e['race']))

0.10254639035622379


In [ ]:
from fairlearn.metrics import demographic_parity_ratio
print(demographic_parity_ratio(y_test_e,
                               y_pred_e,
                               sensitive_features=x_test_e['sex']))

0.17041653450714078


In [ ]:
from fairlearn.metrics import demographic_parity_difference
print(demographic_parity_difference(y_test_e,
                               y_pred_e,
                               sensitive_features=x_test_e['sex']))

0.1265066322975497


In [ ]:
from fairlearn.metrics import equalized_odds_ratio
print(equalized_odds_ratio(y_test_e,
                           y_pred_e,
                           sensitive_features=x_test_e['sex']))

0.134194471266323


In [ ]:
from fairlearn.metrics import equalized_odds_difference
print(equalized_odds_difference(y_test_e,
                           y_pred_e,
                           sensitive_features=x_test_e['sex']))

0.1810685297416116


## Calculation on balanced data below

In [ ]:
from fairlearn.metrics import demographic_parity_ratio
print(demographic_parity_ratio(y_test_b,
                               y_pred_b,
                               sensitive_features=x_test_b['race']))

0.4503118084892375


In [ ]:
from fairlearn.metrics import demographic_parity_difference
print(demographic_parity_difference(y_test_b,
                               y_pred_b,
                               sensitive_features=x_test_b['race']))

0.2983892983892984


In [ ]:
from fairlearn.metrics import equalized_odds_ratio
print(equalized_odds_ratio(y_test_b,
                           y_pred_b,
                           sensitive_features=x_test_b['race']))

0.43229375381130764


In [ ]:
from fairlearn.metrics import equalized_odds_difference
print(equalized_odds_difference(y_test_b,
                           y_pred_b,
                           sensitive_features=x_test_b['race']))

0.32552573760200876


In [ ]:
from fairlearn.metrics import demographic_parity_ratio
print(demographic_parity_ratio(y_test_b,
                               y_pred_b,
                               sensitive_features=x_test_b['sex']))

0.2608257387846028


In [ ]:
from fairlearn.metrics import demographic_parity_difference
print(demographic_parity_difference(y_test_b,
                               y_pred_b,
                               sensitive_features=x_test_b['sex']))

0.4725844743700766


In [ ]:
from fairlearn.metrics import equalized_odds_ratio
print(equalized_odds_ratio(y_test_b,
                           y_pred_b,
                           sensitive_features=x_test_b['sex']))

0.28539896258691094


In [ ]:
from fairlearn.metrics import equalized_odds_difference
print(equalized_odds_difference(y_test_b,
                           y_pred_b,
                           sensitive_features=x_test_b['sex']))

0.5118654235350949


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
x = encoded_data[['workclass','education', 'marital-status', 'occupation', 'relationship',
               'race', 'sex','native-country', 'age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']]
y = encoded_data['income']

x_train_e, x_test_e, y_train_e, y_test_e = train_test_split(x, y, test_size = 0.3, random_state = 108)

smote = SMOTE(sampling_strategy=1, random_state=108)
x_train_smote, y_train_smote = smote.fit_resample(x_train_e, y_train_e)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Create a logistic regression model within a pipeline that includes scaling
model_smote = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, random_state = 108))

# Train the model
model_smote.fit(x_train_smote,y_train_smote)

from sklearn.metrics import accuracy_score, classification_report

# Generate predictions
y_pred_smote = model_smote.predict(x_test_e)

# Evaluate accuracy
accuracy_smote = accuracy_score(y_test_e, y_pred_smote)
print(f"Accuracy: {accuracy_smote}")

# Detailed performance report
report_smote = classification_report(y_test_e, y_pred_smote)
print(report_smote)

Accuracy: 0.7152119019995905
              precision    recall  f1-score   support

           0       0.90      0.70      0.79     11096
           1       0.45      0.75      0.56      3557

    accuracy                           0.72     14653
   macro avg       0.67      0.73      0.68     14653
weighted avg       0.79      0.72      0.73     14653



In [ ]:
from fairlearn.metrics import demographic_parity_ratio
print(demographic_parity_ratio(y_test_e,
                               y_pred_smote,
                               sensitive_features=x_test_e['race']))

0.5555780438004869


In [ ]:
from fairlearn.metrics import demographic_parity_difference
print(demographic_parity_difference(y_test_e,
                               y_pred_smote,
                               sensitive_features=x_test_e['race']))

0.19209719874333264


In [ ]:
from fairlearn.metrics import equalized_odds_ratio
print(equalized_odds_ratio(y_test_e,
                               y_pred_smote,
                               sensitive_features=x_test_e['race']))

0.5321015581980265


In [ ]:
from fairlearn.metrics import equalized_odds_difference
print(equalized_odds_difference(y_test_e,
                               y_pred_smote,
                               sensitive_features=x_test_e['race']))

0.1494445650399999


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
x = encoded_data[['workclass','education', 'marital-status', 'occupation', 'relationship',
                   'native-country', 'age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']]
y = encoded_data['income']

x_train_e_sr, x_test_e_sr, y_train_e_sr, y_test_e_sr = train_test_split(x, y, test_size = 0.3, random_state = 108)

smote = SMOTE(random_state=108)
x_train_smote, y_train_smote = smote.fit_resample(x_train_e_sr, y_train_e_sr)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Create a logistic regression model within a pipeline that includes scaling
model_smote = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, random_state = 108))

# Train the model
model_smote.fit(x_train_smote,y_train_smote)

from sklearn.metrics import accuracy_score, classification_report

# Generate predictions
y_pred_smote = model_smote.predict(x_test_e_sr)

# Evaluate accuracy
accuracy_smote = accuracy_score(y_test_e_sr, y_pred_smote)
print(f"Accuracy: {accuracy_smote}")

# Detailed performance report
report_smote = classification_report(y_test_e_sr, y_pred_smote)
print(report_smote)

Accuracy: 0.7156896198730636
              precision    recall  f1-score   support

           0       0.90      0.70      0.79     11096
           1       0.45      0.76      0.57      3557

    accuracy                           0.72     14653
   macro avg       0.68      0.73      0.68     14653
weighted avg       0.79      0.72      0.73     14653



In [ ]:
import numpy as np
import pandas as pd
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Fetch dataset
adult = fetch_ucirepo(id=2)

# Data (as pandas dataframes)
X_data = adult.data.features
y_data = adult.data.targets

data = pd.concat([X_data, y_data], axis=1)
data[data == '?'] = np.nan
for col in ['workclass', 'occupation', 'native-country']:
    data[col].fillna(data[col].mode()[0], inplace=True)
data['income'] = data['income'].map({'<=50K': 0, '<50K': 0, '<=50K.': 0, '<50K.': 0,
                                     '>50K': 1, '>=50K': 1, '>50K.': 1, '>=50K.': 1})

# Encode categorical variables
categorical = ['workclass', 'education', 'marital-status', 'occupation', 'relationship',
               'race', 'sex', 'native-country']
encoded_data = data.copy()
label_encoder = LabelEncoder()
for col in categorical:
    encoded_data[col] = label_encoder.fit_transform(encoded_data[col])

# Split data into features and target
X = encoded_data.drop('income', axis=1)
y = encoded_data['income']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=108)

# Identify indices of categorical features for SMOTENC (only 'race' and 'sex')
sensitive_indices = [X_train.columns.get_loc(col) for col in ['race', 'sex']]

# Apply SMOTENC to balance the dataset based on 'race' and 'sex'
smotenc = SMOTENC(categorical_features=sensitive_indices, sampling_strategy=1, random_state=108)
X_train_smote, y_train_smote = smotenc.fit_resample(X_train, y_train)




In [ ]:
X_train['sex'].value_counts()

sex
1    22807
0    11382
Name: count, dtype: int64

In [ ]:
X_train_smote['sex'].value_counts()

sex
1    40020
0    12098
Name: count, dtype: int64

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Create a logistic regression model within a pipeline that includes scaling
model_smote = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, random_state = 108))

# Train the model
model_smote.fit(X_train_smote,y_train_smote)

from sklearn.metrics import accuracy_score, classification_report

# Generate predictions
y_pred_smote = model_smote.predict(X_test)

# Evaluate accuracy
accuracy_smote = accuracy_score(y_test, y_pred_smote)
print(f"Accuracy: {accuracy_smote}")

# Detailed performance report
report_smote = classification_report(y_test, y_pred_smote)
print(report_smote)

Accuracy: 0.7644850883778066
              precision    recall  f1-score   support

           0       0.91      0.77      0.83     11096
           1       0.51      0.76      0.61      3557

    accuracy                           0.76     14653
   macro avg       0.71      0.76      0.72     14653
weighted avg       0.81      0.76      0.78     14653



In [ ]:
from fairlearn.metrics import demographic_parity_ratio
print(demographic_parity_ratio(y_test,
                               y_pred_smote,
                               sensitive_features=X_test['sex']))

0.13100329060593963


In [ ]:
from fairlearn.metrics import demographic_parity_difference
print(demographic_parity_difference(y_test,
                               y_pred_smote,
                               sensitive_features=X_test['sex']))

0.4399279287727931


In [ ]:
from fairlearn.metrics import equalized_odds_ratio
print(equalized_odds_ratio(y_test,
                               y_pred_smote,
                               sensitive_features=X_test['sex']))

0.1108434914988333


In [ ]:
from fairlearn.metrics import equalized_odds_difference
print(equalized_odds_difference(y_test,
                               y_pred_smote,
                               sensitive_features=X_test['sex']))

0.5615326305727013
